!TODO:
    
- Explore more complex nor unique preprocessing (singkatan, tidak baku, bahasa inggris, pos tagging)

# Import modules & overview


In [1]:
# basic import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import math

%matplotlib inline

In [2]:
import spacy
import re

In [3]:
link_test = "./dataset/test.csv"

In [4]:
test_df = pd.read_csv(link_test, delimiter=";")

In [78]:
# train_df['text'] = train_df['stemmed_text']
# train_df.drop(columns=['label_angka', 'stemmed_text', 'Unnamed: 0'], inplace=True)

In [5]:
test_df.head()

,IDText,Text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IDText  1000 non-null   object
 1   Text    1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [8]:
test_df.describe()

,IDText,Text
count,1000,1000
unique,1000,1000
top,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
freq,1,1


In [14]:
test_df.rename(columns={"Text":"text"}, inplace=True)

# Text Preprocessing

In [15]:
def counter_word(df, view=10):
    counter = df['text'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in counter for item in sublist])
    temp = pd.DataFrame(top.most_common(view))
    temp.columns = ['Common_words','count']
    return temp.style.background_gradient(cmap='Blues')

def counter_word2(df, view=10, length=5):
    counter = df['text'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in counter for item in sublist if len(item) > length])
    temp = pd.DataFrame(top.most_common(view))
    temp.columns = ['Common_words','count']
    return temp.style.background_gradient(cmap='Blues')

In [16]:
counter_word2(test_df, 30, 11)

,Common_words,count
0,#GanjarMahfud2024,184
1,#GanjarPranowoPilihanUmat,107
2,Ganjar-Mahfud,86
3,#GanjarMahfudRebound,85
4,OposisiCerdas],71
5,#GanjarMahfudM3nangDebat,70
6,kesejahteraan,66
7,#02Melanjutkan,53
8,#3MandatGanjarMahfud,51
9,#GanjarMahfudM3nang,43


In [17]:
process_df = test_df.copy()

## Drop duplicated texts

Data yang duplikat hanya memperberat teks untuk di preproses dan model untuk belajar, belum juga risiko model cenderung bias akibat data duplikat.

In [18]:
process_df.duplicated().sum()

0

In [19]:
process_df.drop_duplicates(inplace=True)

In [20]:
process_df.duplicated().sum()

0

## Removing Unecessary words

Yang termasuk sebagai kata yang tidak penting diantaranya username twitter atau mention, kata yang terlalu panjang

Penjelasan regex:

- @ = Mencocokkan simbol @ sebagai awalan username.
- [A-Za-z0-9+/=]+ =  Mencocokkan satu atau lebih karakter huruf (besar/kecil), angka, tanda plus (+), garis miring (/), atau sama dengan (=).
- (?=\s|$$) =  Menjamin bahwa pola diakhiri dengan spasi (\s) atau akhir baris ($), tanpa meninggalkan karakter setelah username.

In [22]:
# hapus mention @
process_df['text'] = process_df['text'].str.replace(r'@[A-Za-z0-9+/=]+(?=\s|$)', '', regex=True)
process_df.head(10)

,IDText,text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...
3,TXT0004,Prabowo Gibran yang bisa melakukan itu semua ...
4,TXT0005,@uda_zulhendra Lah justru yg gak nyambung ju...
5,TXT0006,Nelayan tak bisa terus terjebak dalam kredit m...
6,TXT0007,Prabowo anti kebebasan pers\n\nhttps://t.co/7S...
7,TXT0008,Kontrak terbuka Ganjar Pranowo adalah cermin d...
8,TXT0009,Ã°ÂŸÂÂ½ Ganjar Mahfud Hebat #Coblos3 #3GM #G...
9,TXT0010,Ganjar-Mahfud adalah harapan baru untuk pertum...


In [23]:
def counter_word_username(df, view=10):
    counter = df['text'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in counter for item in sublist if '@' in item])
    temp = pd.DataFrame(top.most_common(view))
    temp.columns = ['Common_words','count']
    return temp.style.background_gradient(cmap='Blues')

# Top 20 username yang TAG dalam teks training
counter_word_username(process_df, 100)

,Common_words,count
0,@gibran_tweet,11
1,@cuitcuap_,5
2,@Melihat_Indo,5
3,@ajengcute16__,5
4,@bawaslu_RI,3
5,@KPU_ID,3
6,@Resty_J_Cayah,3
7,@Generasi_MudaNU,2
8,@Hasbil_Lbs,2
9,@psi_id,2


Kita bisa melihat bahwa semua username telah bersih, yang tersisa dapat kita hilangkan dengan dictionary sederhana

Penjelasan regex: 

- \[: Mencocokkan tanda kurung siku pembuka [.
- [^\]]+: Mencocokkan satu atau lebih karakter apapun kecuali tanda kurung siku penutup ].
- \]: Mencocokkan tanda kurung siku penutup ].

In [24]:
# hapus username RE
process_df['text'] = process_df['text'].str.replace(r'\[[^\]]+\]', '', regex=True)

In [25]:
def counter_word_username2(df, view=10):
    counter = df['text'].dropna().apply(lambda x: re.findall(r'\[RE ([^\]]+)\]', x)) 
    top = Counter([item for sublist in counter for item in sublist])
    temp = pd.DataFrame(top.most_common(view), columns=['Username', 'Count'])
    return temp.style.background_gradient(cmap='Blues')

counter_word_username2(process_df, 10)

,Username,Count


In [26]:
# hapus link
process_df['text'] = process_df['text'].str.replace(r'https?://\S+', '', regex=True)

In [27]:
# hapus angka dan tanda baca
process_df['text'] = process_df['text'].str.replace(r'[\d!"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~]', '', regex=True)

In [28]:
# hapus kata yang typo lebih dari 20 kata
process_df['text'] = process_df['text'].str.replace(r'\b\w{20,}\b', '', regex=True)

In [29]:
# Menghapus semua karakter non-alfabet dan non-spasi
process_df['text'] = process_df['text'].str.replace(r'[^A-Za-z\s]', '', regex=True)

In [30]:
# Menghapus kata "RT"
process_df['text'] = process_df['text'].str.replace(r'\bRT\b', '', regex=True)

# Mengganti spasi berlebihan dengan satu spasi
process_df['text'] = process_df['text'].str.replace(r'\s{2,}', ' ', regex=True)

In [31]:
process_df.drop_duplicates(inplace=True) # memastikan tidak ada duplikat

Berikut merupakan bruteforce regex filtering kata-kata yang setidaknya tidak digunakan

## Case folding

In [34]:
process_df['text'] = process_df['text'].str.lower()

In [35]:
process_df

,IDText,text
0,TXT0001,lu mau org prodemokrasi di negara ini bisa pun...
1,TXT0002,prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kikidaliyo ganjar pranowo itulah beliau sosok ...
3,TXT0004,prabowo gibran yang bisa melakukan itu semua ...
4,TXT0005,udazulhendra lah justru yg gak nyambung junju...
...,...,...
995,TXT0996,bikin bangga deh ganjarmahfud mau alokasikan s...
996,TXT0997,pak jokowi sebelum pilpres berbesar hati meran...
997,TXT0998,sbaiknya si gemot nga usah ikutan debat dehng...
998,TXT0999,kebiasaan merembuk atau bermusyawarah jadi gay...


In [37]:
print(test_df.loc[0, 'text'])
print()
print(process_df.loc[0, 'text'])

Lu mau org2 pro-demokrasi di negara ini bisa punya kesempatan lagi buat ngelawan oligarki di indo?

- Pilih AMIN, tpi jangan pilih PKS, pilih Partai Buruh
- Jangan pilih Prabowo dan semua partai pengusungnya
- Pilih GAMA, tpi jangan pilih PDIP, pilih Partai Buruh [RE debordsbitch]

lu mau org prodemokrasi di negara ini bisa punya kesempatan lagi buat ngelawan oligarki di indo pilih amin tpi jangan pilih pks pilih partai buruh jangan pilih prabowo dan semua partai pengusungnya pilih gama tpi jangan pilih pdip pilih partai buruh 


## Stopping Stopword

### Sastrawi

Kami menggunakan library Sastrawi untuk stopping stopword, stemming teks Bahasa Indonesia

In [38]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [39]:
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [40]:
stopwords = factory.create_stop_word_remover()

In [41]:
process_df['text'] = process_df['text'].apply(stopwords.remove)

In [42]:
# process_df.duplicated().sum()

0

### Twitter stopwords

https://github.com/Braincore-id/IndoTWEEST

In [43]:
stopwords_df = pd.read_csv('./dictionary/stopwords_twitter.csv')  # Ganti dengan path ke file Anda
stopwords = set(stopwords_df['33books'].str.strip().tolist())  # Mengambil daftar stopword

# Membuat regex untuk mencocokkan stopword
stopword_pattern = r'\b(?:' + '|'.join(map(re.escape, stopwords)) + r')\b'

# Menghapus stopword dari teks
process_df['text'] = process_df['text'].str.replace(stopword_pattern, '', regex=True)

# Menghapus spasi berlebihan (jika diperlukan)
process_df['text'] = process_df['text'].str.replace(r'\s{2,}', ' ', regex=True).str.strip()

### Other stopwords handling

https://github.com/nasalsabila/kamus-alay

In [44]:
slang_df = pd.read_csv('./dictionary/colloquial-indonesian-lexicon.csv')  # Ganti dengan path ke file CSV Anda
slang_dict = dict(zip(slang_df['slang'], slang_df['formal']))  # Membuat dictionary slang -> formal

# Membuat regex pattern untuk semua slang
slang_pattern = r'\b(?:' + '|'.join(map(re.escape, slang_dict.keys())) + r')\b'

# Fungsi untuk mengganti slang dengan formal
def replace_slang(text):
    return re.sub(slang_pattern, lambda match: slang_dict[match.group(0)], text)

# Menerapkan fungsi ke kolom teks
process_df['text'] = process_df['text'].apply(replace_slang)

In [45]:
# process_df.duplicated().sum()
process_df.drop_duplicates(inplace=True) # memastikan tidak ada duplikat

## Stemming text

In [46]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pityudhistira28/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/pityudhistira28/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pityudhistira28/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
df = process_df.copy()

In [49]:
def tokenize_text(sentence):
    token = word_tokenize(sentence)
    return token

In [50]:
df['token'] = df['text'].apply(tokenize_text)
df.head()

,IDText,text,token
0,TXT0001,mau orang prodemokrasi negara bisa punya kesem...,"[mau, orang, prodemokrasi, negara, bisa, punya..."
1,TXT0002,prabowo ditanya soal hutang luar negeri menjaw...,"[prabowo, ditanya, soal, hutang, luar, negeri,..."
2,TXT0003,kikidaliyo ganjar pranowo beliau sosok mengagu...,"[kikidaliyo, ganjar, pranowo, beliau, sosok, m..."
3,TXT0004,prabowo gibran bisa melakukan semua kesejahter...,"[prabowo, gibran, bisa, melakukan, semua, kese..."
4,TXT0005,udazulhendra lah justru menyambung junjungan l...,"[udazulhendra, lah, justru, menyambung, junjun..."


In [51]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

In [52]:
def stemming_text(tokens):
    result = [stemmer.stem(token) for token in tokens]
    return result

In [53]:
df['stemmed'] = df['token'].apply(stemming_text)

In [54]:
df['stemmed_text'] = df['stemmed'].apply(lambda tokens: ' '.join(tokens))
df

,IDText,text,token,stemmed,stemmed_text
0,TXT0001,mau orang prodemokrasi negara bisa punya kesem...,"[mau, orang, prodemokrasi, negara, bisa, punya...","[mau, orang, prodemokrasi, negara, bisa, punya...",mau orang prodemokrasi negara bisa punya sempa...
1,TXT0002,prabowo ditanya soal hutang luar negeri menjaw...,"[prabowo, ditanya, soal, hutang, luar, negeri,...","[prabowo, tanya, soal, hutang, luar, negeri, j...",prabowo tanya soal hutang luar negeri jawab hu...
2,TXT0003,kikidaliyo ganjar pranowo beliau sosok mengagu...,"[kikidaliyo, ganjar, pranowo, beliau, sosok, m...","[kikidaliyo, ganjar, pranowo, beliau, sosok, m...",kikidaliyo ganjar pranowo beliau sosok mengagu...
3,TXT0004,prabowo gibran bisa melakukan semua kesejahter...,"[prabowo, gibran, bisa, melakukan, semua, kese...","[prabowo, gibran, bisa, laku, semua, sejahtera...",prabowo gibran bisa laku semua sejahtera rakyat
4,TXT0005,udazulhendra lah justru menyambung junjungan l...,"[udazulhendra, lah, justru, menyambung, junjun...","[udazulhendra, lah, justru, sambung, junjung, ...",udazulhendra lah justru sambung junjung lu aom...
...,...,...,...,...,...
995,TXT0996,bangga ganjarmahfud mau alokasikan setengah be...,"[bangga, ganjarmahfud, mau, alokasikan, seteng...","[bangga, ganjarmahfud, mau, alokasi, tengah, b...",bangga ganjarmahfud mau alokasi tengah belanja...
996,TXT0997,pak jokowi pilpres berbesar hati merangkul pak...,"[pak, jokowi, pilpres, berbesar, hati, merangk...","[pak, jokowi, pilpres, besar, hati, rangkul, p...",pak jokowi pilpres besar hati rangkul pak prab...
997,TXT0998,sebaiknya sih gemot enggak usah ikutan debat d...,"[sebaiknya, sih, gemot, enggak, usah, ikutan, ...","[baik, sih, got, enggak, usah, ikut, debat, de...",baik sih got enggak usah ikut debat dehnga jel...
998,TXT0999,kebiasaan merembuk bermusyawarah jadi gaya kep...,"[kebiasaan, merembuk, bermusyawarah, jadi, gay...","[biasa, rembuk, musyawarah, jadi, gaya, pimpin...",biasa rembuk musyawarah jadi gaya pimpin ganja...


In [55]:
process_df['text'] = df['stemmed_text']

In [56]:
process_df.duplicated().sum()

0

## Typo dictionary

In [58]:
robin_df = pd.read_csv('./dictionary/robin_dict.csv')  # Ganti dengan path ke file CSV Anda
robin_dict = dict(zip(robin_df['text lama'], robin_df['text baru']))  

# Ganti nilai NaN dengan string kosong dan pastikan semua nilai adalah string
process_df['text'] = process_df['text'].fillna('').astype(str)

# Membuat regex pattern untuk semua slang
robin = r'\b(?:' + '|'.join(map(re.escape, robin_dict.keys())) + r')\b'

# Fungsi untuk mengganti slang dengan formal
def replace_slang(text):
    return re.sub(robin, lambda match: robin_dict[match.group(0)], text)

# Menerapkan fungsi ke kolom teks
process_df['text'] = process_df['text'].apply(replace_slang)

In [59]:
process_df

,IDText,text
0,TXT0001,mau orang prodemokrasi negara bisa punya sempa...
1,TXT0002,prabowo tanya soal hutang luar negeri jawab hu...
2,TXT0003,kikidaliyo ganjar pranowo beliau sosok mengagu...
3,TXT0004,prabowo gibran bisa laku semua sejahtera rakyat
4,TXT0005,udazulhendra lah justru sambung junjung lu aom...
...,...,...
995,TXT0996,bangga Ganjar-Mahfud mau alokasi tengah belanj...
996,TXT0997,pak jokowi pilpres besar hati rangkul pak prab...
997,TXT0998,baik sih got enggak usah ikut debat dehnga jel...
998,TXT0999,biasa rembuk musyawarah jadi gaya pimpin ganja...


# Export

In [60]:
process_df.to_csv('./dataset/test_clean.csv', index=False)